In [36]:
import torch
import torchaudio

In [ ]:
def preprocess(filepath):
    # resampling, to mono
    resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=22050)
    wf, sr = torchaudio.load(filepath)
    wf = resample(wf)
    wf = torch.mean(wf, dim=0)

    # transform to spectrogram,
    make_spectrogram = torchaudio.transforms.Spectrogram(n_fft=510, hop_length=259, power=None, return_complex=True)
    input_spec = make_spectrogram(wf)

    # split in real and imaginary numbers
    input_tensor = torch.empty(size=list([2] + list(input_spec.shape)))
    input_tensor[0] = input_spec.real
    input_tensor[1] = input_spec.imag

    # create batch for input in neural network
    input_batch = torch.split(input_tensor, 256, dim=-1)
    if ((input_tensor.shape[-1]%256) != 0):
        input_batch = torch.stack(input_batch[:-1])
    else:
        input_batch = torch.stack(input_batch)
    
    return input_batch

In [39]:
# load input file and transform to mono, 22.050Hz

filepath = 'witchfucker-dunkel.wav'

resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=22050)
def to_mono(waveform): return torch.mean(waveform, dim=0)

wf, sr = torchaudio.load(filepath)
wf_ = resample(wf)
wf_ = to_mono(wf_)


In [150]:
# transform to spectrogram, create input tensor

make_spectrogram = torchaudio.transforms.Spectrogram(n_fft=510, hop_length=259, power=None, return_complex=True)

# spectrogram with complex numbers
input_spec = make_spectrogram(wf_)

# split in real and imaginary numbers
input_tensor = torch.empty(size=list([2] + list(input_spec.shape)))
input_tensor[0] = input_spec.real
input_tensor[1] = input_spec.imag

# create batch for input in neural network


In [152]:
input_batch = torch.split(input_tensor, 256, dim=-1)

if ((input_tensor.shape[-1]%256) != 0):
    input_batch = torch.stack(input_batch[:-1])
else:
    input_batch = torch.stack(input_batch)

In [153]:
input_batch.shape

torch.Size([47, 2, 256, 256])